##**Bibliotecas**

In [2]:
import pandas as pd
from pandas.tseries.offsets import BDay
from datetime import date
pd.set_option("display.max_colwidth", 150)

###**Funções para Coleta e Tratamento de dados de Fundos de Investimento fornecidos pela CVM**

In [7]:
def busca_cadastro_cvm(data=(date.today()-BDay(60))):
  if data is not busca_cadastro_cvm.__defaults__[0]:
    data = pd.to_datetime(data)
  
  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv'.format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')
 
  except: 
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")

In [8]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS') 
  informe_completo = pd.DataFrame()
 
  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';')    
    
    except: 
      print("Arquivo {} não encontrado!".format(url))    
 
    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)
 
  return informe_completo

In [15]:
def melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')  
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
 
  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
 
  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
 
  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
 
  #piores
  piores = pd.DataFrame()
  piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:top] - 1) * 100
  for cnpj in piores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    piores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    piores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores, piores

In [9]:
def consulta_fundo(informes, cnpj):  
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0]
  return fundo

###**Buscando Dados da CVM**

In [10]:
cadastro = busca_cadastro_cvm()

In [11]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2020-01', data_fim='2020-05')

In [12]:
cadastro

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,36.017.639/0001-27,051 7010 FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO CP INVEST NO EXTERIOR,2020-06-05,2020-01-06,NaN,FASE PRÉ-OPERACIONAL,2020-01-06,NaN,NaN,NaN,Fundo Multimercado,2020-01-06,DI de um dia,Fechado,S,N,S,S,20.0,NaN,0.40,NaN,NaN,NaN,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A
1,34.172.417/0001-53,051 AÇÕES FUNDO DE INVESTIMENTO DE AÇÕES,2019-08-22,2019-08-20,NaN,EM FUNCIONAMENTO NORMAL,2019-08-22,2019-08-22,2020-07-01,2021-06-30,Fundo de Ações,2019-08-20,NaN,Aberto,N,N,NaN,S,0.0,NaN,0.10,NaN,1.187080e+08,2020-11-27,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,36.016.411/0001-12,051 ALLOCATION FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO,2020-04-03,2020-04-02,NaN,EM FUNCIONAMENTO NORMAL,2020-06-16,2020-06-16,2020-07-01,2021-06-30,Fundo Multimercado,2020-04-02,NaN,Aberto,S,N,S,S,0.0,NaN,0.10,NaN,1.482105e+07,2020-11-27,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
3,36.498.260/0001-86,051 ALOCAÇÃO AÇÕES FUNDO DE INVESTIMENTO DE AÇÕES,2020-04-02,2020-04-02,NaN,FASE PRÉ-OPERACIONAL,2020-04-02,NaN,NaN,NaN,Fundo de Ações,2020-07-06,NaN,Aberto,N,N,NaN,S,0.0,NaN,0.20,NaN,0.000000e+00,2020-11-27,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
4,36.498.567/0001-87,051 ALOCAÇÃO MODERADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO,2020-04-13,2020-04-13,NaN,EM FUNCIONAMENTO NORMAL,2020-06-16,2020-06-16,2020-10-01,2021-09-30,Fundo Multimercado,2020-04-13,NaN,Aberto,S,N,S,S,10.0,O fundo cobrara 10.00 % do que exceder 100.00 % o indice CDIE. A taxa de performance sera paga no(s) mes(es) de Janeiro e Julho,0.70,NaN,2.485149e+06,2020-11-27,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39468,27.389.638/0001-12,ZURICH SCHRODER GROWTH FUNDO DE INVESTIMENTO MULTIMERCADO PREVIDENCIÁRIO,2018-08-07,2017-01-21,NaN,FASE PRÉ-OPERACIONAL,2017-01-21,NaN,NaN,NaN,Fundo Multimercado,2017-01-21,OUTROS,Aberto,N,S,N,S,0.0,NaN,1.00,NaN,NaN,NaN,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE 

In [13]:
informes

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2020-01-02,1132491.66,27.225023,1123583.00,0.0,0.0,1
1,00.017.024/0001-53,2020-01-03,1132685.12,27.224496,1123561.25,0.0,0.0,1
2,00.017.024/0001-53,2020-01-06,1132881.43,27.225564,1123605.31,0.0,0.0,1
3,00.017.024/0001-53,2020-01-07,1133076.85,27.226701,1123652.24,0.0,0.0,1
4,00.017.024/0001-53,2020-01-08,1132948.59,27.227816,1123698.26,0.0,0.0,1
...,...,...,...,...,...,...,...,...
1751290,97.929.213/0001-34,2020-05-25,64115760.27,8.435821,64106518.16,0.0,0.0,2
1751291,97.929.213/0001-34,2020-05-26,64033683.92,8.424972,64024066.62,0.0,0.0,2
1751292,97.929.213/0001-34,2020-05-27,64408421.03,8.474234,64398427.06,0.0,0.0,2
1751293,97.929.213/0001-34,2020-05-28,64337663.50,8.464873,64327293.16,0.0,0.0,2


###**Fundos de Investimento: 10 Melhores e 10 Piores de 2020**

In [16]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100)

In [ ]:
melhores

In [ ]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
24.837.071/0001-01,-75.899511,WM DELTA CRÉDITO ESTRUTURADO FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO - CRÉDITO PRIVADO,Fundo Multimercado,1.107619e+06
26.648.868/0001-96,-63.445553,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,3.411718e+08
12.987.743/0001-86,-63.377084,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.371619e+09
31.936.857/0001-41,-62.783939,TREND IBOVESPA ALAVANCADO FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,8.549544e+07
10.869.628/0001-81,-56.136318,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,1.815607e+07
10.590.125/0001-72,-55.825129,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,5.942560e+06
11.182.072/0001-13,-53.975565,LOGOS TOTAL RETURN FI EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,1.507532e+08
32.319.324/0001-83,-53.694348,TREND IBOVESPA DOLAR FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,7.348693e+06
10.551.375/0001-01,-50.176998,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA CONSTRUÇÃO CIVIL,Fundo de Ações,3.788286e+08


###**Fundos de Ações: 10 Melhores e 10 Piores de 2020**

In [17]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='acoes')

In [18]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
19.831.126/0001-36,32.916286,WESTERN ASSET FUNDO DE INVESTIMENTO AÇÕES BDR NÍVEL I,Fundo de Ações,2.284556e+09
13.199.100/0001-30,30.453904,SANTANDER FUNDO DE INVESTIMENTO COLABORADORES MAGAZINE LUIZA AÇÕES,Fundo de Ações,3.996441e+07
17.503.172/0001-80,25.018572,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA BDR NÍVEL I,Fundo de Ações,5.644914e+08
34.028.082/0001-02,24.468864,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I PLUS,Fundo de Ações,1.701688e+08
22.632.237/0001-28,23.986235,BB AÇÕES GLOBAIS FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,5.256385e+08
13.401.224/0001-57,23.985009,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR NÍVEL I,Fundo de Ações,2.910446e+08
19.436.818/0001-80,20.689707,SAFRA CONSUMO AMERICANO FUNDO DE INVESTIMENTO EM AÇÕES BDR-NÍVEL I PB,Fundo de Ações,1.786749e+09
19.436.835/0001-17,20.169119,SAFRA CONSUMO AMERICANO FDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES BDR-NÍVEL I,Fundo de Ações,1.189113e+09
21.347.643/0001-86,20.106906,BRADESCO FIC DE FUNDOS DE INVESTIMENTO EM AÇÕES GLOBAL INVESTIMENTO NO EXTERIOR,Fundo de Ações,1.385842e+08


In [19]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
26.648.868/0001-96,-58.658495,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES II - BDR NÍVEL I,Fundo de Ações,4.467671e+08
12.987.743/0001-86,-58.628503,ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I,Fundo de Ações,1.784249e+09
31.936.857/0001-41,-56.473327,TREND IBOVESPA ALAVANCADO FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,9.399194e+07
10.869.628/0001-81,-52.266591,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,2.453526e+07
10.590.125/0001-72,-51.921386,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,6.470649e+06
10.292.302/0001-34,-48.087959,HAYP FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,1.101639e+08
32.319.324/0001-83,-46.849821,TREND IBOVESPA DOLAR FUNDO DE INVESTIMENTO EM AÇÕES,Fundo de Ações,1.370100e+07
10.601.479/0001-75,-44.410988,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES BRADESCO,Fundo de Ações,5.175679e+07
10.551.375/0001-01,-42.342952,FUNDO DE INVESTIMENTO EM AÇÕES CAIXA CONSTRUÇÃO CIVIL,Fundo de Ações,5.139334e+08


###**Fundos de Renda Fixa: 10 Melhores e 10 Piores de 2020**

In [20]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='rendafixa')

In [21]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
30.606.402/0001-03,33.647380,BB RENDA FIXA LONGO PRAZO BONDS GLOBAIS FX PRIVATE CRÉDITO PRIVADO INVESTIMENTO NO EXTERIOR FIC FI,Fundo de Renda Fixa,7.715308e+07
00.360.293/0001-18,32.205712,BB RENDA FIXA DIVIDA EXTERNA MIL FUNDO INVESTIMENTO,Fundo de Renda Fixa,2.859834e+08
26.751.304/0001-84,28.983356,BB RENDA FIXA LP CRÉDITO PRIVADO FX BONDS BRASILEIROS INVESTIMENTO NO EXTERIOR PRIVATE FIC FI,Fundo de Renda Fixa,4.091499e+07
04.877.280/0001-71,14.091167,SANTOS CREDIT YIELD FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,4.654179e+07
04.240.128/0001-83,10.915954,BV CRÉDITO ATIVO RENDA FIXA CRÉDITO PRIVADO LONGO PRAZO FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,5.471781e+07
03.737.207/0001-31,9.160906,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAIXA SELEÇÃO RENDA FIXA,Fundo de Renda Fixa,1.408730e+07
06.114.630/0001-73,8.454572,SANTOS IV LQ FUNDO DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,2.306749e+06
04.570.232/0001-36,8.299181,MB MATIC RENDA FIXA LONGO PRAZO FUNDO DE INVESTIMENTO FINANCEIRO,Fundo de Renda Fixa,2.160644e+06
26.344.495/0001-60,6.197274,ITAÚ PRIVATE RENDA FIXA PRÉ IDKA 3 FICFI,Fundo de Renda Fixa,1.049890e+08


In [22]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
01.671.480/0001-85,-32.754005,SANTOS VIRTUAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CRÉDITO PRIVADO,Fundo de Renda Fixa,5.768385e+06
05.357.507/0001-10,-10.366968,BANESTES INSTITUCIONAL FUNDO DE INVESTIMENTO RENDA FIXA,Fundo de Renda Fixa,2.931408e+08
31.095.989/0001-98,-8.260326,ITAÚ RENDA FIXA JUROS REAIS B5+ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,7.656063e+08
07.400.556/0001-14,-8.153515,ICATU VANGUARDA FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO INFLAÇÃO LONGA RF LP,Fundo de Renda Fixa,3.228189e+08
27.884.230/0001-17,-8.049262,ITAÚ PRIVATE RENDA FIXA IMA-B 5+ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo de Renda Fixa,1.711816e+08
10.264.241/0001-00,-7.903658,ITAÚ FLEXPREV VÉRTICE IMAB5 MAIS RENDA FIXA - FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,4.557276e+09
10.577.503/0001-88,-7.903428,FUNDO DE INVESTIMENTO CAIXA BRASIL IMA B 5+ TITULOS PÚBLICOS RENDA FIXA LONGO PRAZO,Fundo de Renda Fixa,1.746167e+09
20.374.752/0001-20,-7.856033,BTG PACTUAL TESOURO IPCA LONGO FUNDO DE INVESTIMENTO RENDA FIXA REFERENCIADO,Fundo de Renda Fixa,2.992158e+08
13.327.340/0001-73,-7.828900,BB PREVIDENCIÁRIO RENDA FIXA IMA-B5+ TÍTULOS PÚBLICOS FUNDO DE INVESTIMENTO,Fundo de Renda Fixa,1.794897e+09


###**Fundos Cambiais: 10 Melhores de 10 Piores de 2020**

In [23]:
melhores, piores = melhores_e_piores(informes, cadastro, top=10, minimo_de_cotistas=100, classe='cambial')

In [24]:
melhores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
33.144.687/0001-98,34.238679,BTG PACTUAL DIGITAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.113136e+08
01.623.535/0001-81,33.986018,ITAÚ CAMBIAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,8.422901e+08
31.132.386/0001-19,33.919504,TREND DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,2.897674e+08
01.430.938/0001-04,33.825920,BTG PACTUAL DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.470361e+08
02.294.024/0001-26,33.823079,BRADESCO H FUNDO DE INVESTIMENTO CAMBIAL LONGO PRAZO DÓLAR,Fundo Cambial,2.256255e+08
28.516.058/0001-01,33.772301,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL ÁGORA,Fundo Cambial,1.863465e+07
23.731.760/0001-74,33.729513,ITAÚ PERSONNALITÉ CAMBIAL DÓLAR PLUS FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,9.358097e+07
23.711.486/0001-71,33.646045,OCCAM FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,5.311036e+07
02.498.204/0001-20,33.585709,SANTANDER FIC FI TOP CAMBIAL,Fundo Cambial,1.255835e+08


In [25]:
piores

,retorno(%),Fundo de Investimento,Classe,PL
CNPJ_FUNDO,,,,
32.246.789/0001-51,-25.073773,TREND SHORT DOLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.346294e+07
00.819.855/0001-49,30.507353,WESTERN ASSET CÂMBIO DOLAR SILVER FICFI CAMBIAL,Fundo Cambial,2.496242e+06
00.819.880/0001-22,31.999773,WESTERN ASSET CÂMBIO DOLAR PRIME FICFI CAMBIAL,Fundo Cambial,3.621237e+07
00.819.889/0001-33,32.040607,WESTERN ASSET HEDGE DÓLAR FUNDO DE INVESTIMENTO CAMBIAL,Fundo Cambial,1.005106e+08
01.171.956/0001-19,32.700422,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL DÓLAR,Fundo Cambial,1.111056e+07
02.543.933/0001-50,32.884467,SANTANDER FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO CAMBIAL,Fundo Cambial,7.013568e+07
04.128.893/0001-06,32.945294,BB CAMBIAL DOLAR LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,2.629153e+08
04.061.044/0001-82,33.049286,BB CAMBIAL DÓLAR LP VIP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,4.663192e+08
04.305.193/0001-40,33.226015,BB CAMBIAL EURO LP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,Fundo Cambial,9.788266e+07
